---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [78]:
import networkx as nx
import pandas as pd
import numpy as np
import re
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [79]:
def answer_one():
        
    # Your Code Here
    df = pd.DataFrame(pd.read_csv('Employee_Movie_Choices.txt'))
    df = df['#Employee\tMovie'].str.split('\t')
    df2 = pd.DataFrame()
    df2['Employee'] = df.apply(lambda x: x[0])
    df2['Movie'] = df.apply(lambda x: x[1])
    
    G = nx.from_pandas_dataframe(df2, 'Employee', 'Movie')
    
    #print(bipartite.is_bipartite(G))
    #plot_graph(G)
    
    return G

answer_one()

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [80]:
def answer_two():
    
    # Your Code Here
    G = answer_one()
    
    employees = pd.Series(list(pd.Series(bipartite.sets(G))[0]))
    employees.apply(lambda x: G.add_node(x, type='employee'))
    
    employees = pd.Series(list(pd.Series(bipartite.sets(G))[1]))
    employees.apply(lambda x: G.add_node(x, type='movie'))
    
    return G

answer_two()

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [81]:
def answer_three():
        
    G = answer_two()
    X = bipartite.sets(G)[0]
    
    P = bipartite.weighted_projected_graph(G, X)
    
    #print(P.edges(data=True))
    #plot_graph(P, 'weight')
    
    return P

answer_three()

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [82]:
def combine(x, G1, G2):
    try: 
        G2.add_edge(x[0],x[1],
                    weight=G1[x[0]][x[1]]['weight'])
    except:
        G2.add_edge(x[0],x[1],
                    weight=0)

In [83]:
def answer_four():
        
    # Your Code Here
    G1 = answer_three()
    G1_edges = pd.Series(G1.edges(data=True))

    df = pd.DataFrame(pd.read_csv('Employee_Relationships.txt',
                                  names = ['Emp1','Emp2','rel'],
                                  header = None,
                                  delimiter='\t'))

    G2 = nx.from_pandas_dataframe(df, 'Emp1', 'Emp2', 'rel')
    
    G2_edges = pd.Series(G2.edges(data=True))
    
    G2_edges.apply(lambda x : combine(x, G1, G2))
      
    G2_edges = pd.Series(G2.edges(data=True))
    
    df = pd.DataFrame()
    df['rel'] = G2_edges.apply(lambda x: x[2]['rel']) 
    df['weight'] = G2_edges.apply(lambda x: x[2]['weight'])

    return df['rel'].corr(df['weight'])

answer_four()

0.78839622217334748